In [1]:
from pyspark.sql import SparkSession
# Initialze Spark Session
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1679618890476_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark.sql.functions as F
from re import sub
from pyspark.ml.feature import MinMaxScaler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Read in metadata table.
index = spark \
        .read \
        .option("header", True) \
        .csv("s3://w251-asl-data/raw-data/train.csv")

index.show(3, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------+--------------+-----------+-----+
|path                                         |participant_id|sequence_id|sign |
+---------------------------------------------+--------------+-----------+-----+
|train_landmark_files/26734/1000035562.parquet|26734         |1000035562 |blow |
|train_landmark_files/28656/1000106739.parquet|28656         |1000106739 |wait |
|train_landmark_files/16069/100015657.parquet |16069         |100015657  |cloud|
+---------------------------------------------+--------------+-----------+-----+
only showing top 3 rows

In [ ]:
from pyspark.sql.types import DoubleType

min_x = -1.33437180519104
max_x = 2.9205052852630615
min_y = -2.301203727722168
max_y = 3.572496175765991

def scale(val, min_, max_):
    return (val - min_) / (max_ - min_)

scale_x_udf = F.udf(lambda z: scale(z, min_x, max_x), DoubleType())
scale_y_udf = F.udf(lambda z: scale(z, min_y, max_y), DoubleType())

# Read in raw data, manipulate to include filename as join key, join to index table, drop any rows missing data. 
data = spark.read.parquet("s3://w251-asl-data/raw-data/train_landmark_files/*/*") \
        .na.drop() \
        .withColumn("scaled_x", scale_x_udf(F.col("x"))) \
        .drop("x") \
        .withColumn("scaled_y", scale_x_udf(F.col("y"))) \
        .drop("y") \
        .withColumn("path", F.input_file_name()) \
        .withColumn("path", F.substring("path", 29, 1000)) \
        .join(index, ["path"], "left") \
        .drop(F.col("z")) \
        .withColumn("landmark", F.concat(F.col("type"), F.lit("-"), F.col("landmark_index"))) \
        .drop("path", "row_id", "type", "landmark_index") \
        .groupBy(["participant_id", "sequence_id", "frame"]) \
        .pivot("landmark").sum("scaled_x", "scaled_y") \
        .fillna(0) \
        .cache()
        
data.show(3, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data.dtypes

In [ ]:
data.describe().show()

In [ ]:
# Get max frame length
data.select(F.max(F.col("frame"))).show()